In [3]:
import os, base64

directory="images"
index="000000"

image_data = []
for folder in os.listdir(directory):
    if not folder.startswith("cam"): continue
    for filename in os.listdir(f"{directory}/{folder}"):
        if filename.startswith(index) and "Distance" not in filename:
            with open(f"{directory}/{folder}/{filename}", 'rb') as f:
                image_data.append(base64.b64encode(f.read()).decode('utf-8'))

In [ ]:
from dsta_mvs.dsta_mvs.mvs_utils.metadata_reader import MetadataReader

directory="images"
metadata = MetadataReader(directory)
metadata.read_metadata_and_initialize_dirs(f"{directory}/metadata.json", f"{directory}/frame_graph.json")
print(metadata.frame_graph)

In [ ]:
import json
import torch
from dsta_mvs.dsta_mvs.mvs_utils.camera_models import ShapeStruct, LinearSphere

f = open('images/manifest.json')
manifest = json.load(f)
f.close()

print(manifest['camera_models']['cam_model_0'])
linear_model = LinearSphere(fov_degree=195, shape_struct=ShapeStruct(1000, 1000))
coord_batch = torch.rand((1, 2, 1)).to('cuda')

# Get rays for these coordinates.
rays, _ = linear_model.pixel_2_ray(coord_batch)
print(rays)

# Project the rays.
pix, _ = linear_model.point_3d_2_pixel(rays)
print(pix)

In [ ]:
import os
import sys

# The path of the current Python script.
_CURRENT_PATH = os.path.dirname(os.path.realpath(__file__))
_TOP_PATH     = os.path.join(_CURRENT_PATH, 'dsta_mvs')

if _TOP_PATH not in sys.path:
    sys.path.insert( 0, _TOP_PATH)
    for i, p in enumerate(sys.path):
        print(f'{i}: {p}')

from api.dataset_player import DatasetProxy
import torch

dataset_player = DatasetProxy("./config.py", "./images")

transformations = dict()
camera_frame = dataset_player.dataset.map_camera_frame
print(camera_frame)
cameras = [v for k, v in camera_frame.items() if 'cam' in k]
for i in range(len(cameras)):
    for j in range(len(cameras)):
        if i != j:
            transformations[(cameras[i], cameras[j])] = dataset_player.dataset.frame_graph.query_transform(f0=cameras[i], f1=cameras[j])
# print(transformations)
camera_models = dict([(k, v.__dict__) for k, v in dataset_player.dataset.map_camera_model_raw.items()]) #raw or not raw
print(camera_models)
